In [9]:
DATASET_URL = "https://gist.githubusercontent.com/YHYeoh/ad1a7f7170c72d621d05a70637540152/raw/5a6059c199e2c46d2f3d258f03d93cfea98e2749/marketing_campaign.csv"
data = pd.read_csv(DATASET_URL, sep = ';')

pd.set_option('plotting.backend','pandas_bokeh')

# data.fillna(method = "ffill", inplace = True)
data.isnull().values.any()


education_order = [['Basic', 'Graduation', 'Master', '2n Cycle', 'PhD']]
ordinal_encoder = OrdinalEncoder(categories=education_order)
enc = OneHotEncoder()


imr = IterativeImputer(random_state=42, max_iter=100, min_value=data['Income'].min())
imr = imr.fit(data[['Income']])
data['Income'] = imr.transform(data[['Income']]).ravel()


data["Education"] = (ordinal_encoder.fit_transform(data["Education"].values.reshape(-1, 1))).astype(int)
print(ordinal_encoder.categories_)

data_copy = data.copy()
marital_status_ohe = pd.get_dummies(data["Marital_Status"],prefix="Marital")
ohe_cols = marital_status_ohe.columns
data = pd.concat([data, marital_status_ohe], axis=1)

data.drop(["ID", 'Dt_Customer',"Z_CostContact","Z_Revenue","Marital_Status"], axis=1, inplace=True)

categorical = ['Marital_Status']
numerical = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain']
numerical_no_bool = ['Education','Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

y = data.Response
print(y.value_counts())
X = data.drop(['Response'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.3)

X_train_cont = X_train.drop(['AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2','Education','Complain','Year_Birth','Marital_Absurd', 'Marital_Alone', 'Marital_Divorced',
       'Marital_Married', 'Marital_Single', 'Marital_Together',
       'Marital_Widow', 'Marital_YOLO'],axis=1)

X_test_cont = X_test.drop(['AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2','Education','Complain','Year_Birth','Marital_Absurd', 'Marital_Alone', 'Marital_Divorced',
       'Marital_Married', 'Marital_Single', 'Marital_Together',
       'Marital_Widow', 'Marital_YOLO'],axis=1)
X_cont_column = X_train_cont.columns
pcaX_train = X_train_cont
pcaX_test = X_test_cont

[array(['Basic', 'Graduation', 'Master', '2n Cycle', 'PhD'], dtype=object)]
0    1906
1     334
Name: Response, dtype: int64


In [10]:

def getIterativeImputedIncome(data):
	imr = IterativeImputer(random_state=42, max_iter=100, min_value= data['Income'].min())
	imr = imr.fit(data[['Income']])
	data['Income'] = imr.transform(data[['Income']]).ravel()
	return data

def getKNNImputedIncome(data):
	imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
	data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
	return data

def getAgeFromDateBirth(data):
	year = datetime.datetime.now().year
	data['Year_Birth'] = data['Year_Birth'].apply(lambda x : year - x )
	return data

def extractFromDate(data):
	data['enroll_year'] = pd.DatetimeIndex(data.Dt_Customer).year
	data['enroll_month'] = pd.DatetimeIndex(data.Dt_Customer).month
	data['enroll_day'] = pd.DatetimeIndex(data.Dt_Customer).day
	data.drop(['Dt_Customer'], axis = 1, inplace= True)
	return data

def convertToDays(data):
	vfunc = np.vectorize(lambda x: (datetime.datetime.now() - x).days)
	data['Dt_Customer'] = vfunc(pd.DatetimeIndex(data.Dt_Customer).to_pydatetime())
	return data

def getBinnedIncome(data):
	data['Income'] = (pd.cut(data['Income'], bins=[0, 15000, 60000, 110000, 300000], labels=False, precision=0)).convert_dtypes()
	return data

def getNormalizedIncome(data):
	data = data[(np.abs(stats.zscore(data[['Income']])) < 3)]
	return data

def getNormalizedAndBinnedIncome(data):
	data = data[(np.abs(stats.zscore(data[['Income']])) < 3)]
	data['Income'] = pd.cut(data['Income'], bins=[0, 15000, 60000, 110000, 300000], labels=False, precision=0).convert_dtypes()
	return data

def getHypeYearBirth(data):
	hyper_year_birth = [
		data,
		getAgeFromDateBirth(data.copy()),
	]
	return hyper_year_birth



def getPreprocessingIncome(data):
	preprocessing_income = [
		getIterativeImputedIncome(data.copy()),
		getKNNImputedIncome(data.copy()),
		data.copy().fillna(method = "ffill"), #ffill
		data.copy().fillna(method = "bfill"), #bfill
		data.copy().fillna(data.mean()), #mean imputed
		data.copy().fillna(data.median()), #median imputed
		data.copy().dropna(subset=['Income'])
	]
	return preprocessing_income

def getHypeIncome(data):
	hyper_income = [
	data,
	getBinnedIncome(data.copy()),
	getNormalizedIncome(data.copy()),
	getNormalizedAndBinnedIncome(data.copy())
	]
	return hyper_income

In [ ]:
data['SalaryRange']

In [11]:
def hasmethod(obj, name):
	return inspect.ismethod(getattr(obj, name, None))


In [12]:

def setupPreprocessPipeline(scaler):
	ss = Pipeline(steps=[('scaler',scaler)])
	#ohe = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown = 'ignore'))])
	preprocess = ColumnTransformer(
                    transformers=[
                        ('cont', ss, numerical_no_bool)
                        #('cat', ohe, categorical),
                        #('le', le, ordinal),
                        ],remainder='passthrough')
	return preprocess


In [13]:
def feature_importance(classifier, feature_names, scaler_name):
	if (hasattr(classifier,'coef_')):
		importance = classifier.coef_[0]
	elif (hasattr(classifier,'coefs_')):
		importance = classifier.coefs_
	elif (hasattr(classifier,'feature_importances_')):
		importance = classifier.feature_importances_
	else:
		print("Cannot extract feature importance, skipping")
		return

	#for i,v in enumerate(importance):
	#	print('Feature: %d, Score: %.5f' % (i,v))
	zipped = zip(feature_names, importance)
	df = pd.DataFrame(zipped, columns=["feature", "value"])
	# Sort the features by the absolute value of their coefficient
	df["abs_value"] = df["value"].apply(lambda x: abs(x))
	df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
	df = df.sort_values("abs_value", ascending=False)
	# plot feature importance
	fig, ax = plt.subplots(1, 1, figsize=(16, 9))
	sns.barplot(x="feature",
	            y="value",
	            data=df.head(20),
	           palette=df.head(20)["colors"])
	plt.gcf().subplots_adjust(bottom=0.30)
	ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=14)
	ax.set_title("Top 20 Features for {} w/ {}".format(classifier.__class__.__name__, scaler_name), fontsize=25)
	ax.set_ylabel("Coef", fontsize=22)
	ax.set_xlabel("Feature Name", fontsize=22)
	plt.show()


In [14]:

    
def overallClassificationReport(model,classes):
	visualizer = ClassificationReport(model, classes=classes, support=True) #might can change
	visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
	if(model.__class__.__name__ == "XGBClassifier"): #special treatment for xgboost as it reordered column
		model.fit(X_train, y_train)
		reorderedColumn = model.get_booster().feature_names
		reordered_Xtest = X_test[reorderedColumn] #reorderColumn
		visualizer.score(X_test, y_test)        
		visualizer.show()
		return
	visualizer.score(X_test, y_test)        # Evaluate the model on the test data
	visualizer.show()


def overall_feature_importance(model,X_train,y_train):
	labels = list(map(lambda s: s.title(), X.columns))
	viz = FeatureImportances(model, labels=labels,encoder={1: 'yes',0: 'no'}, relative=False, topn = 8)
	viz.fit(X, y)
	viz.show()

def has_feature_imp(classifier):
	status = False
	if (hasattr(classifier,'coef_')):
		status = True
	elif (hasattr(classifier,'coefs_')):
		status = True
	elif (hasattr(classifier,'feature_importances_')):
		status = True
	print("Cannot extract feature importance, skipping")
	return status


In [15]:



# def pcaComparison(scaler,X_train,y_train,X_test,y_test):
#     cv=10
#     scaler = scaler
#     X_train_scaled = scaler.fit_transform(pcaX_train,y_train)
#     X_test_scaled = scaler.transform(pcaX_test)
#     pcaPerformance = pd.DataFrame()
#     for transformer in pcaParamGrid :
#         pca = transformer['transformer']
#         param_grid =  transformer['param_grid']
#         PCASearch = RandomizedSearchCV(pca,param_grid,n_iter=10,verbose=2,
#                          scoring= scorer,
#                          n_jobs=-1,cv=cv,random_state=42)
#         PCASearch = PCASearch.fit(X_train_scaled,y_train)

#         param = PCASearch.best_params_
#         name = PCASearch.best_estimator_.__class__.__name__
#         score = PCASearch.best_score_
#         pcaResult = {"Model":name,"MSE": score,"Parameter":param,"scaler":scaler}
#         pcaPerformance = pcaPerformance.append(pcaResult,ignore_index=True)
#     pcaPerformance.sort_values(by='MSE',ascending=True,inplace=True)
#     return pcaPerformance

In [16]:
# pcaParamGrid = [{
#     'transformer':PCA(),'param_grid':{'n_components':list(range(2,8)),'random_state':[42]
#                                       ,'whiten':[True,False]
#                                       ,'svd_solver':['full', 'arpack', 'randomized']}
# },{
#     'transformer':KernelPCA(),
#     'param_grid':{'gamma':loguniform(1e-6, 100),'n_components':list(range(2,8))
#                   ,'random_state':[42],'kernel':['rbf'],
#                   'alpha':loguniform(1e-6, 100),'eigen_solver':['dense', 'arpack']
#                   ,'n_jobs':[-1],'max_iter':list(range(1,1000)),'fit_inverse_transform':[True,False]}
# },
#     {
#         'transformer':KernelPCA(),
#     'param_grid':{'gamma':loguniform(1e-6, 100),'n_components':list(range(2,8))
#                   ,'random_state':[42],'kernel':['poly'],
#                   'alpha':loguniform(1e-6, 100),'degree':list(range(3,8))
#                   ,'n_jobs':[-1],'max_iter':list(range(1,1000)),'eigen_solver':['dense', 'arpack'],
#                  'fit_inverse_transform':[True,False]}
#     }
# ]


In [17]:
# differentScalerResult = pd.DataFrame()
# scalers = [StandardScaler(),MinMaxScaler(),MaxAbsScaler(), RobustScaler(),QuantileTransformer()]

# for scaler in scalers:
#     pcaResult = pcaComparison(scaler,pcaX_train,y_train,pcaX_test,y_test)
#     differentScalerResult = differentScalerResult.append(pcaResult,ignore_index=True)
# differentScalerResult.sort_values(by='MSE',ascending=True,inplace=True)


In [18]:
# differentScalerResult

In [19]:
# #use the same scaler same as pca
# scaler = MaxAbsScaler()
# print(scaler)
# scaledXTrain = pd.DataFrame(scaler.fit_transform(pcaX_train),columns=X_cont_column)
# scaledXTest = pd.DataFrame(scaler.transform(pcaX_test),columns=X_cont_column)

# km_list = list()
# for clust in range(2,21):
#     km = KMeans(n_clusters=clust, random_state=42)
#     km = km.fit(scaledXTrain)
#     y_pred = km.predict(scaledXTrain)
#     sh = silhouette_score(scaledXTrain,y_pred)
#     km_list.append(pd.Series({'clusters': clust, 
#                               'inertia': km.inertia_,
#                               'model': km,
#                              'sil':sh}))
# io.reset_output()
# io.output_notebook()


# plot_data = (pd.concat(km_list, axis=1)
#              .T
#              [['clusters','inertia','sil']]
#              .set_index('clusters'))
# plot_dataInertia=plot_data['inertia'].astype(float)
# ax = plot_dataInertia.plot(marker='o')

# #concat so show sihhoutte score
# plot_dataSil=plot_data['sil'].astype(float)
# ax = plot_dataSil.plot(marker='o')


In [20]:
# #saw 6 cluster has a good silhouette score, create kmean column with 6 cluster
# km = KMeans(n_clusters=6, random_state=42)
# km = km.fit(scaledXTrain)
# kmeanTrainColumn = pd.Series(km.predict(scaledXTrain), name="KMeansTrain")
# kmeanTestColumn = pd.Series(km.predict(scaledXTest), name="KMeansTest")

In [21]:
# kmeanTrainColumn

In [22]:
# i=1
# for rowNo,result in differentScalerResult.iterrows():
#     mse,model,param,scaler = result
#     print(str(i)+" "+str(model))
#     print(mse)
#     print(param)
#     print(scaler)
#     print("\n")
#     if(i==5):
#         break
#     i+=1


In [23]:
# #KernelPCA has the lowest error
# kpca = KernelPCA(alpha=7.577453045410569, degree= 7, eigen_solver= "dense"
#                  , fit_inverse_transform= True, gamma= 1.2767906371238508e-06
#                  , kernel= "poly", max_iter= 601, n_components= 5, n_jobs= -1, random_state= 42)
# scaler= MaxAbsScaler()
# X_train_scaled = scaler.fit_transform(pcaX_train,y_train)
# X_test_scaled = scaler.transform(pcaX_test)

# kpca = kpca.fit(X_train_scaled,y_train)
# transformedXTrain = kpca.transform(X_train_scaled)
# transformedXTest=kpca.transform(X_test_scaled)


In [24]:
# explained_variance = np.var(transformedXTrain, axis=0)
# explained_variance_ratio = explained_variance / np.sum(explained_variance)
# print("Explained Variance : \n" +str(explained_variance))
# print("Explained Variance Ratio: \n"+str(explained_variance_ratio))

In [25]:
# kpcaCol = ["PCA1","PCA2","PCA3","PCA4","PCA5"]
# kpcaTrain = pd.DataFrame(transformedXTrain,columns = kpcaCol)
# kpcaTest = pd.DataFrame(transformedXTest, columns = kpcaCol)
# #reset all train test columns to ensure consistency before merging
# X_train = X_train.reset_index()
# X_test = X_test.reset_index()
# y_train = y_train.reset_index()
# y_test = y_test.reset_index()
# #drop all continuous columns

# X_train_Kmeans = X_train.copy()
# X_test_Kmeans = X_test.copy()
# X_train_PCA_Kmeans = X_train.drop(columns=X_cont_column,axis=1)
# X_test_PCA_Kmeans = X_test.drop(columns=X_cont_column,axis=1)


In [26]:
#remove default unimportant column
unimportant = ["Year_Birth"]
# X_train_Kmeans = X_train_Kmeans.drop(columns=unimportant,axis=1)
# X_test_Kmeans = X_test_Kmeans.drop(columns=unimportant,axis=1)
# X_train_PCA_Kmeans = X_train_PCA_Kmeans.drop(columns=unimportant,axis=1)
# X_test_PCA_Kmeans = X_test_PCA_Kmeans.drop(columns=unimportant,axis=1)
X_train = X_train.drop(columns=unimportant,axis=1)
X_test = X_test.drop(columns=unimportant,axis=1)
y_train = y_train.drop(columns=['index'],axis=1)
y_test = y_test.drop(columns=['index'],axis=1)

In [27]:
y_train

994     0
2162    0
906     1
572     0
1877    0
       ..
1638    0
1095    0
1130    0
1294    0
860     0
Name: Response, Length: 1568, dtype: int64

In [28]:
# #add kmean column into all known data except x_train
# X_train_Kmeans["KMean"]=kmeanTrainColumn
# X_test_Kmeans["KMean"]=kmeanTestColumn
# X_train_PCA_Kmeans["KMean"]=kmeanTrainColumn
# X_test_PCA_Kmeans["KMean"]=kmeanTestColumn

In [29]:
# X_train_PCA_Kmeans

In [30]:
# #addPCA column
# kpcaCol = ["PCA1","PCA2","PCA3","PCA4","PCA5"]

# #PCA and Kmeans
# X_train_PCA_Kmeans[kpcaCol]=kpcaTrain
# X_test_PCA_Kmeans[kpcaCol]=kpcaTest

# #PCA only
# X_train_PCA = kpcaTrain
# X_test_PCA = kpcaTest

In [31]:
# rbf_sampler = RBFSampler(gamma = 5.0,n_components=100)
# X_train_rbf = rbf_sampler.fit_transform(X_train_PCA)
# X_test_rbf = rbf_sampler.transform(X_test_PCA)

In [32]:
# X_trainS, y_trainS = sampler.fit_resample(X_train, y_train.copy())
# # stratified(SMOTE) x with k means
# X_train_Kmeans, y_trainR = sampler.fit_resample(X_train_Kmeans, y_train.copy())
# # stratified(SMOTE) x with pca
# X_train_PCA, y_trainR = sampler.fit_resample(X_train_PCA, y_train.copy())
# # stratified(SMOTE) x with pca and k means
# X_train_PCA_Kmeans, y_trainR = sampler.fit_resample(X_train_PCA_Kmeans, y_train.copy())

In [33]:
# sampler = SMOTE()
# stratified(SMOTE) x
# stratified(SMOTE) x with k means
# stratified(SMOTE) x with pca
# stratified(SMOTE) x with pca and k means

#X_train, y_train.copy()
#X_train_Kmeans, y_train.copy()

oversampler = RandomOverSampler(sampling_strategy=0.5)
undersampler = RandomUnderSampler(sampling_strategy='majority')
X_trainOV,y_trainOV = oversampler.fit_resample(X_train, y_train)
X_trainUN, trainUN = undersampler.fit_resample(X_train, y_train)

NameError: name 'RandomOverSampler' is not defined

In [ ]:
dataList = [
    {
        'DataName':'Unmodified Dataset','X_train':X_train,'y_train':y_train
    },
    {
        'DataName':'Oversampled Dataset','X_train':X_trainOV,'y_train':y_trainOV
    },
    {
        'DataName':'Undersampled Dataset','X_train':X_trainOV,'y_train':y_trainOV
    }
]
model_result = []

In [ ]:
def cross_validate(classifier, cv,datasetName,X_train,y_train):
	scalers = [StandardScaler(),MinMaxScaler(),MaxAbsScaler(), RobustScaler(),QuantileTransformer()]
	train_acc = []
	test_acc = []
	mean = []
	result = []
	for scaler in scalers:
		fpr = None
		tpr = None
		preprocess = setupPreprocessPipeline(scaler)
		pipeline = Pipeline(steps=[
	        ('preprocess', preprocess),
	        ('classifier', classifier)
		])

		train_acc = []
		test_acc = []
		train_recall = []
		test_recall = []
		train_precision = []
		test_precision = []
		train_f1 = []
		test_f1 = []
		mean = []
		
		for train_ind, val_ind in cv.split(X_train, y_train):
			X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
			pipeline.fit(X_t, y_t)
			y_hat_t = pipeline.predict(X_t)
			train_acc.append(accuracy_score(y_t, y_hat_t))
			train_recall.append(recall_score(y_t, y_hat_t))
			train_precision.append(precision_score(y_t, y_hat_t))
			train_f1.append(f1_score(y_t, y_hat_t))
			X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind] 
			y_hat_val = pipeline.predict(X_val)
			test_acc.append(accuracy_score(y_val, y_hat_val))
			test_recall.append(recall_score(y_val, y_hat_val))
			test_precision.append(precision_score(y_val, y_hat_val))
			test_f1.append(f1_score(y_val, y_hat_val))

		#ohe_cols = list(pipeline.named_steps['preprocess'].named_transformers_['cat'].named_steps['ohe'].get_feature_names(input_features=categorical))
		numeric_feature_list = list(numerical)
		for i in ohe_cols:
			numeric_feature_list.append(i)
		#print(len(numeric_feature_list))
		#evaluation(y_val, y_hat_val, 'Confusion Matrix {} + {}'.format(classifier.__class__.__name__, scaler.__class__.__name__).strip())
		print('Mean Training Accuracy: {} | Standard Deviation: {}'.format(np.mean(train_acc),np.std(test_acc)))
		print('Mean Validation Accuracy: {} | Standard Deviation: {}'.format(np.mean(test_acc),np.std(test_acc)))
		print('\n')
# 		feature_importance(classifier, numeric_feature_list, scaler.__class__.__name__ )
		print(y_val.shape,y_hat_val.shape)
		# 
# 		if hasmethod(pipeline['classifier'], 'predict_proba'):
# 			fpr,tpr = ROC_Curve_Plot(pipeline,X_val,y_val,classifier.__class__.__name__ +" w "+scaler.__class__.__name__)
		model_result.append({
            'classifier':classifier.__class__.__name__,
            'scalerName':scaler.__class__.__name__,
            'datasetName':datasetName,
            'train_accuracy':np.mean(train_acc),
            'test_accuracy':np.mean(test_acc),
            'train_recall':np.mean(train_recall),
            'test_recall':np.mean(test_recall),
            'train_precision':np.mean(train_precision),
            'test_precision':np.mean(test_precision),
            'train_f1':np.mean(train_f1),
            'test_f1':np.mean(test_f1)
        })
	return result

In [ ]:
feature_importance

In [ ]:
# models = [
# 	#Lasso(),
# # 	SGDClassifier(max_iter = 1000, tol=1e-3,penalty = "elasticnet"),
# # 	LinearSVC(), 
# # 	GaussianProcessClassifier(),
# # 	ExtraTreesClassifier(), 
# # 	BernoulliNB(),
# 	LogisticRegressionCV(max_iter= 1200), 
# # 	RidgeClassifierCV(),
# 	SVC(kernel = 'linear',max_iter= -1), 
# # 	Perceptron(),
# # 	PassiveAggressiveClassifier(), 
# # 	DecisionTreeClassifier(), #no coef 
# # 	KNeighborsClassifier(),#no feat_import, use permutation_importance 
# # 	GaussianNB(), #no feat_import, use permutation_importance 
# 	LGBMClassifier(),#no coef 
# # 	RandomForestClassifier(), #no coef 
# # 	GradientBoostingClassifier(),#no coef 
# # 	PassiveAggressiveClassifier(), 
# # 	ExtraTreesClassifier(), #no coef 
# 	XGBClassifier(),
# 	AdaBoostClassifier(), #no coef
# # 	MLPClassifier() #mlp not working
# 	]



models = [
	#Lasso(),
# 	SGDClassifier(max_iter = 1000, tol=1e-3,penalty = "elasticnet"),
# 	LinearSVC(), 
# 	GaussianProcessClassifier(),
# 	ExtraTreesClassifier(), 
# # 	BernoulliNB(),
# 	LogisticRegressionCV(max_iter= 1200), 
# # 	RidgeClassifierCV(),
# 	SVC(kernel = 'linear',max_iter= -1), 
# 	Perceptron(),
# 	PassiveAggressiveClassifier(), 
# 	DecisionTreeClassifier(), #no coef 
# 	KNeighborsClassifier(),#no feat_import, use permutation_importance 
# 	GaussianNB(), #no feat_import, use permutation_importance 
# 	LGBMClassifier(),#no coef 
	RandomForestClassifier(), #no coef 
# 	GradientBoostingClassifier(),#no coef 
# 	PassiveAggressiveClassifier(), 
# 	ExtraTreesClassifier(), #no coef 
# 	XGBClassifier(),
# 	AdaBoostClassifier(), #no coef
# 	MLPClassifier() #mlp not working
	]


In [ ]:
model_result= []
classes = ["no", "yes"]

for data in dataList:
    print(data['DataName'])
    xTrainData = data['X_train']
    yTrainData = data['y_train']
    for model in models:
        print(model.__class__.__name__)
        model_result.append(cross_validate(model,StratifiedKFold(),data['DataName'],xTrainData, yTrainData))
        overallClassificationReport(model,classes)
        if hasmethod(model, 'predict_proba'):
            ROC_Curve_Plot(model,X_test,y_test,"Overall "+model.__class__.__name__)
        if has_feature_imp(model) :
            overall_feature_importance(model,xTrainData, yTrainData)

import warnings
warnings.filterwarnings("default")

In [ ]:
resultDF = pd.DataFrame.from_records(model_result,columns=['classifier','scalerName','datasetName','train_accuracy','test_accuracy','train_recall','test_recall','train_precision','test_precision','train_f1','test_f1'])

In [ ]:
resultDF.dropna(how='all')

In [ ]:
# DATASET_URL = "https://gist.githubusercontent.com/YHYeoh/ad1a7f7170c72d621d05a70637540152/raw/5a6059c199e2c46d2f3d258f03d93cfea98e2749/marketing_campaign.csv"
# data = pd.read_csv(DATASET_URL, sep = ';')

# pd.set_option('plotting.backend','pandas_bokeh')

# # data.fillna(method = "ffill", inplace = True)
# data.isnull().values.any()


# education_order = [['Basic', 'Graduation', 'Master', '2n Cycle', 'PhD']]
# ordinal_encoder = OrdinalEncoder(categories=education_order)
# enc = OneHotEncoder()


# imr = IterativeImputer(random_state=42, max_iter=100, min_value=data['Income'].min())
# imr = imr.fit(data[['Income']])
# data['Income'] = imr.transform(data[['Income']]).ravel()


# data["Education"] = (ordinal_encoder.fit_transform(data["Education"].values.reshape(-1, 1))).astype(int)
# print(ordinal_encoder.categories_)

# data_copy = data.copy()
# marital_status_ohe = pd.get_dummies(data["Marital_Status"],prefix="Marital")
# ohe_cols = marital_status_ohe.columns
# data = pd.concat([data, marital_status_ohe], axis=1)

# data.drop(["ID", 'Dt_Customer',"Z_CostContact","Z_Revenue","Marital_Status"], axis=1, inplace=True)

# categorical = ['Marital_Status']
# numerical = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
#        'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
#        'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
#        'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
#        'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
#        'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
#        'Complain']
# numerical_no_bool = ['Education','Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

